<a href="https://colab.research.google.com/github/MaxHenk/2022_ML_EES/blob/main/ML_PROJECT_MH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Projet pour le cours de Machine Learning for Earth & Environmental Sciences.

In [ ]:
#Import Libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [ ]:
#Data Preprocessing
